In [1]:
import pandas as pd 
import numpy as np
import sklearn as sklearn
import os as os

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import re as re

if os.getlogin()=="JVARGH7":
    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'

elif os.getlogin()=='aamnasoniwala':
    path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

# path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
# path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

excel_path = path_equity_precision_llm_folder + "/llm training/Unclassified Splits/Unattributable Part "
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")

In [2]:
excel_path_unattributable = path_equity_precision_llm_folder + "/llm training/epldat03_Unattributable Data.csv"
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/clean_input.py")
execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")
execfile(path_equity_precision_llm_repo + "/functions/standardize_population.py")
execfile(path_equity_precision_llm_repo + "/functions/adjusted_source_population_match.py")

In [3]:
input_unattributable = pd.read_csv(excel_path_unattributable)
input_unattributable.head()

,PMID,MeSH,Abstract,Title,year
0,18008027,"Albuminuria, Angiotensin II Type 1 Receptor Bl...",The purpose of the present study was to invest...,Renoprotective effects of telmisartan on renal...,2013
1,18205795,"Antigens, Tumor-Associated, Carbohydrate, Bloo...",CA 72-4 is one of the blood group carbohydrate...,CA 72-4 levels in patients with type 2 diabete...,2011
2,18378359,"Age Distribution, Age Factors, Aged, 80 and ov...",The aim of this study was to evaluate the asso...,Vascular risk factors and dementia in the gene...,2010
3,18626579,"Bioreactors, Biotechnology, Fermentation, Gluc...",The production of propionic acid by Propioniba...,Kinetic analysis and pH-shift control strategy...,2010
4,18651247,"Bioreactors, Butyric Acid, Cells, Immobilized,...",Butyric acid has many applications in chemical...,Production of butyric acid from glucose and xy...,2010


In [6]:
import pandas as pd
import re

n_json_splits = 11
# Concatenate multiple CSVs
results = None
for scenario in range(1, n_json_splits + 1):
    file_path = f"{path_equity_precision_llm_folder}/llm training/Unclassified Splits/Unattributable Part {scenario}_results.csv"
    temp_df = pd.read_csv(file_path, na_values=['n/a','NaN'], keep_default_na=False)
    temp_df = temp_df[['pmid', 'title', 'gpt_precision_medicine', 'gpt_diabetes', 'gpt_primary_study', 'gpt_source_population']]

    if results is None:
        results = temp_df
    else:
        results = pd.concat([results, temp_df], ignore_index=True)

# Merge datasets
merged_df_unattributable = input_unattributable.merge(results, left_on='PMID', right_on='pmid', how='left')
merged_df_unattributable['gpt_source_population'] = merged_df_unattributable['gpt_source_population'].apply(standardize_population)

merged_df_unattributable = merged_df_unattributable.drop_duplicates(subset='PMID', keep='first').reset_index(drop=True)
merged_df_unattributable = merged_df_unattributable[merged_df_unattributable['Abstract'].notna()].reset_index(drop=True)
merged_df_unattributable = merged_df_unattributable[
    merged_df_unattributable['gpt_precision_medicine'].notna() &
    (merged_df_unattributable['gpt_precision_medicine'] != 'no (insufficient information)')
].reset_index(drop=True)


merged_df_unattributable.to_csv(f"{path_equity_precision_llm_folder}/llm training/epl05_merged unattributable data.csv", index=False)
merged_df_unattributable.shape

(125923, 11)

In [7]:
rows, columns = merged_df_unattributable.shape
print(f"Number of rows: {rows}")
print(f"Number of columns: {columns}")

Number of rows: 125923
Number of columns: 11


In [8]:
unique_source_populations = merged_df_unattributable['gpt_source_population'].unique()
print(unique_source_populations)

['Unknown' 'we' 'ea' 'na' 'mena' 'cee'
 'ea, ssa, sa, we, na, cee, mena, seap, ca, lac' 'sa' 'ssa' 'na, lac'
 'lac' 'seap' 'ea, sa' 'ea, we' 'we, ssa' 'sa, ssa, we' 'ssa, we' 'na, we'
 'lac, ssa' 'sa, we' 'lac, mena, we' 'sa, na' 'ssa, we, ea, lac' 'ea, na'
 'ea, sa, we' 'mena, sa' 'we, na' 'na, we, seap' 'we, sa'
 'we, ea, seap, lac' 'ea, seap' 'ssa, na' 'cee, we' 'lac, mena' 'ca'
 'ca, mena, ea, we' 'na, ea, ssa, lac' 'na, seap, lac, ssa, we'
 'na, cee, we' 'na, ssa' 'ea, ssa' 'we, ea' 'seap, we' 'we, mena'
 'we, ssa, lac, mena' 'we, na, lac' 'na, lac, ssa, we' 'mena, we'
 'we, sa, ssa' 'na, ea, we' 'na, lac, we, ea'
 'na, ea, sa, ssa, we, lac, cee, seap, ca, mena' 'na, mena, sa'
 'seap, ea, sa, we' 'na, we, ssa' 'we, sa, mena' 'lac, we'
 'ea, sa, we, ssa' 'cee, na' 'ea, cee' 'ea, lac' 'ea, cee, we, na'
 'cee, ea, lac, ssa' 'ea, aa, sa'
 'ea, ssa, we, na, cee, seap, sa, ca, mena, lac' 'na, ea, we, sa'
 'we, cee, ea, lac' 'mena, we, sa' 'ca, cee, ea, lac, mena, seap, sa, ssa'
 'na, ea

In [9]:
# Check the output below
merged_df_unattributable.head()

,PMID,MeSH,Abstract,Title,year,pmid,title,gpt_precision_medicine,gpt_diabetes,gpt_primary_study,gpt_source_population
0,18008027,"Albuminuria, Angiotensin II Type 1 Receptor Bl...",The purpose of the present study was to invest...,Renoprotective effects of telmisartan on renal...,2013,18008027.0,Renoprotective effects of telmisartan on rena...,no,yes,yes,Unknown
1,18205795,"Antigens, Tumor-Associated, Carbohydrate, Bloo...",CA 72-4 is one of the blood group carbohydrate...,CA 72-4 levels in patients with type 2 diabete...,2011,18205795.0,CA 72-4 levels in patients with type 2 diabet...,no,yes,yes,Unknown
2,18378359,"Age Distribution, Age Factors, Aged, 80 and ov...",The aim of this study was to evaluate the asso...,Vascular risk factors and dementia in the gene...,2010,18378359.0,Vascular risk factors and dementia in the gen...,no,yes,yes,we
3,18626579,"Bioreactors, Biotechnology, Fermentation, Gluc...",The production of propionic acid by Propioniba...,Kinetic analysis and pH-shift control strategy...,2010,18626579.0,Kinetic analysis and pH-shift control strateg...,no,no,yes,Unknown
4,18651247,"Bioreactors, Butyric Acid, Cells, Immobilized,...",Butyric acid has many applications in chemical...,Production of butyric acid from glucose and xy...,2010,18651247.0,Production of butyric acid from glucose and x...,no,no,yes,Unknown


In [10]:
import pandas as pd

# Define the valid source population categories
VALID_REGIONS = {"ca", "na", "cee", "lac", "sa", "ssa", "we", "seap", "mena", "ea", "Unknown"}
EUROPEAN_REGIONS = {"we", "na"}
NON_EUROPEAN_REGIONS = VALID_REGIONS - EUROPEAN_REGIONS - {"Unknown"}

# Step 1: Identify and count excluded studies due to missing Title or Abstract
def is_missing(value):
    return pd.isna(value) or str(value).strip() == ""

def excluded(row):
    # return is_missing(row['Title']) or is_missing(row['Abstract']) or is_missing(row['MeSH'])
    return is_missing(row['Title']) or is_missing(row['Abstract'])

# Count excluded studies
excluded_studies_df = merged_df_unattributable[merged_df_unattributable.apply(excluded, axis=1)]
excluded_count = len(excluded_studies_df)
print(f"Number of studies excluded due to missing Abstract: {excluded_count}")

# Apply exclusion
merged_df_unattributable_after_exclusion = merged_df_unattributable.drop(excluded_studies_df.index)

# Verify remaining studies
remaining_studies = len(merged_df_unattributable_after_exclusion)
print(f"Remaining studies after exclusion: {remaining_studies}")

merged_df_unattributable_after_exclusion.shape


Number of studies excluded due to missing Abstract: 0
Remaining studies after exclusion: 125923


(125923, 11)

In [ ]:
NON_EUROPEAN_REGIONS

In [11]:
total_classified_and_unclassified = remaining_studies + excluded_count
print("remaining: " + str(remaining_studies) + "; excluded: " + str(excluded_count) + "; total: " + str(total_classified_and_unclassified))


remaining: 125923; excluded: 0; total: 125923


#### @Aamna: The below should reflect the number of abstracts that have more than 1 region attached to it.


In [12]:
# Step 2: Normalize source population categories

def normalize_source_population(source_population):
    if pd.isna(source_population):
        return set()
    return VALID_REGIONS.intersection(set(re.sub(r'\s+', '', source_population).split(',')))

merged_df_unattributable_after_exclusion['normalized_regions'] = merged_df_unattributable_after_exclusion['gpt_source_population'].apply(normalize_source_population)

# Count rows where normalized_regions has more than 1 element
multi_region_rows = merged_df_unattributable_after_exclusion[merged_df_unattributable_after_exclusion['normalized_regions'].apply(lambda x: len(x) > 1)]
multi_region_count = len(multi_region_rows)
print(f"Number of rows where normalized_regions has more than 1 element: {multi_region_count}")

Number of rows where normalized_regions has more than 1 element: 929


In [13]:
# Step 3: Expand data **only** for regional counts
expanded_rows = []
for _, row in merged_df_unattributable_after_exclusion.iterrows():
    for region in row['normalized_regions']:
        new_row = row.copy()
        new_row['gpt_source_population'] = region
        expanded_rows.append(new_row)

# Create expanded DataFrame for regional counts
expanded_df = pd.DataFrame(expanded_rows)
expanded_df.shape

(127576, 12)

In [14]:
expanded_df.head(n=20)

,PMID,MeSH,Abstract,Title,year,pmid,title,gpt_precision_medicine,gpt_diabetes,gpt_primary_study,gpt_source_population,normalized_regions
0,18008027,"Albuminuria, Angiotensin II Type 1 Receptor Bl...",The purpose of the present study was to invest...,Renoprotective effects of telmisartan on renal...,2013,18008027.0,Renoprotective effects of telmisartan on rena...,no,yes,yes,Unknown,{Unknown}
1,18205795,"Antigens, Tumor-Associated, Carbohydrate, Bloo...",CA 72-4 is one of the blood group carbohydrate...,CA 72-4 levels in patients with type 2 diabete...,2011,18205795.0,CA 72-4 levels in patients with type 2 diabet...,no,yes,yes,Unknown,{Unknown}
2,18378359,"Age Distribution, Age Factors, Aged, 80 and ov...",The aim of this study was to evaluate the asso...,Vascular risk factors and dementia in the gene...,2010,18378359.0,Vascular risk factors and dementia in the gen...,no,yes,yes,we,{we}
3,18626579,"Bioreactors, Biotechnology, Fermentation, Gluc...",The production of propionic acid by Propioniba...,Kinetic analysis and pH-shift control strategy...,2010,18626579.0,Kinetic analysis and pH-shift control strateg...,no,no,yes,Unknown,{Unknown}
4,18651247,"Bioreactors, Butyric Acid, Cells, Immobilized,...",Butyric acid has many applications in chemical...,Production of butyric acid from glucose and xy...,2010,18651247.0,Production of butyric acid from glucose and x...,no,no,yes,Unknown,{Unknown}
5,18800199,"5-Aminolevulinate Synthetase, Aminolevulinic A...",The 5-aminolevulinate (ALA) synthase gene (hem...,Expression of a hemA gene from Agrobacterium r...,2010,18800199.0,Expression of a hemA gene from Agrobacterium ...,no,no,yes,Unknown,{Unknown}
6,18809217,"Adipocytes, Animals, Blood Glucose, Body Weigh...",Tumor necrosis factor-alpha (TNF-alpha) is an ...,Pioglitazone reduces tumor necrosis factor-alp...,2010,18809217.0,Pioglitazone reduces tumor necrosis factor-al...,no,no,yes,Unknown,{Unknown}
7,18817993,"Aged, Biomarkers, Blood Glucose, Coronary Arte...",A large clinical trial clarified that pioglita...,Pioglitazone induces regression of coronary at...,2010,18817993.0,Pioglitazone induces regression of coronary a...,no,yes,yes,we,{we}
8,18840627,"Adult, Child, DNA-Binding Proteins, Endoplasmi...",Endoplasmic reticulum stress is a central feat...,Relationship between functional promoter polym...,2010,18840627.0,Relationship between functional promoter poly...,yes,no,yes,ea,{ea}
9,18922591,"Adiponectin, Biomarkers, Blood Glucose, Corona...",Because of the strong association between abdo...,The presence of abdominal obesity is associate...,2010,18922591.0,The presence of abdominal obesity is associat...,no,no,yes,na,{na}


In [15]:
# Step 4: Create a crosstab summary function
def crosstab_summary(expanded_df, categories, region_col='gpt_source_population'):
    expanded_df['combined_category'] = expanded_df[categories].astype(str).agg('_'.join, axis=1)
    summary = pd.crosstab(expanded_df['combined_category'], expanded_df[region_col])
    summary.reset_index(inplace=True)
    return summary

# Define categories
categories = ['gpt_precision_medicine', 'gpt_diabetes', 'gpt_primary_study']

# Generate crosstab for regional counts
summary_expanded_df = crosstab_summary(expanded_df, categories, region_col='gpt_source_population')

# Step 5: Compute overall totals without double-counting studies
total_counts = {
    'Total Precision Medicine': merged_df_unattributable_after_exclusion['gpt_precision_medicine'].notna().sum(),
    'Total Diabetes': merged_df_unattributable_after_exclusion['gpt_diabetes'].notna().sum(),
    'Total Primary Study': merged_df_unattributable_after_exclusion['gpt_primary_study'].notna().sum()
}

total_counts

{'Total Precision Medicine': 125923,
 'Total Diabetes': 125923,
 'Total Primary Study': 125923}

In [16]:
# Step 6: Save the expanded output for regions
output_path = path_equity_precision_llm_repo + '/preprocessing/epl05_combined_output_Unattributable.csv'
summary_expanded_df.to_csv(output_path, index=False)

# Display summary
display(summary_expanded_df)

gpt_source_population,combined_category,Unknown,ca,cee,ea,lac,mena,na,sa,seap,ssa,we
0,no_no_no,7099,5,75,337,52,58,5001,41,31,23,374
1,no_no_yes,22122,29,1400,2931,523,1142,10464,476,218,284,3304
2,no_yes_no,2763,6,106,460,34,91,4538,63,25,34,528
3,no_yes_yes,12919,18,1523,3778,571,1382,11150,961,357,428,4047
4,yes_no_no,312,0,11,48,3,6,531,1,1,14,115
5,yes_no_yes,1668,13,314,939,100,197,2366,91,40,81,1169
6,yes_yes_no,755,3,40,111,15,20,1309,20,10,29,311
7,yes_yes_yes,2776,36,632,2168,210,507,5120,253,141,155,3164


In [17]:
# Step 7: Categorize studies into European vs. Non-European vs. Unknown (without duplicates)
def classify_region(regions):
    classification = set()
    if any(r in EUROPEAN_REGIONS for r in regions):
        classification.add('European')
    if any(r in NON_EUROPEAN_REGIONS for r in regions):
        classification.add('Non-European')
    if not classification:
        classification.add('Unknown')
    return classification

# Remove rows where gpt_source_population is 'Unknown'


merged_df_unattributable_after_exclusion['classification_group'] = merged_df_unattributable_after_exclusion['normalized_regions'].apply(classify_region)

merged_df_unattributable_after_exclusion_region = merged_df_unattributable_after_exclusion[merged_df_unattributable_after_exclusion['gpt_source_population'] != 'Unknown']

# Compute total counts per classification group
group_totals = merged_df_unattributable_after_exclusion_region['classification_group'].value_counts()


In [18]:

expanded_rows_region = []
for _, row in merged_df_unattributable_after_exclusion_region.iterrows():
    for group in row['classification_group']:
        new_row = row.copy()
        new_row['classification_group'] = group
        expanded_rows_region.append(new_row)

# Create expanded DataFrame for regional counts
expanded_df_region = pd.DataFrame(expanded_rows_region)
expanded_df_region.shape


summary_expanded_df_region = crosstab_summary(expanded_df_region, categories, region_col='classification_group')


In [19]:
unknown_classification_rows = expanded_df_region[expanded_df_region['classification_group'] == 'Unknown']
unknown_classification_rows.to_csv(path_equity_precision_llm_folder + '/llm training/epl05_expanded rows region with unknown classification group.csv', index=False)

### We need to figure out why there are still Unknowns!
@aamna: there shouldn't be unknowns at this stage

In [21]:
# Step 8: Save the expanded output for regions
output_path_group = path_equity_precision_llm_repo + '/preprocessing/epl05_combined_output_Unattributable by group.csv'
summary_expanded_df_region.to_csv(output_path_group, index=False)

# Display summary
display(summary_expanded_df_region)

classification_group,combined_category,European,Non-European
0,no_no_no,5325,567
1,no_no_yes,13736,6890
2,no_yes_no,5019,745
3,no_yes_yes,15143,8809
4,yes_no_no,639,81
5,yes_no_yes,3506,1741
6,yes_yes_no,1596,200
7,yes_yes_yes,8182,3940


In [22]:
merged_df_unattributable_after_exclusion_region = merged_df_unattributable_after_exclusion[
    merged_df_unattributable_after_exclusion['classification_group'].apply(lambda x: 'Unknown' not in x)
]
print("Remaining studies after filtering out Unknown:")
print(merged_df_unattributable_after_exclusion_region.shape)


Remaining studies after filtering out Unknown:
(75509, 13)


In [23]:
merged_df_unattributable_after_exclusion.to_csv(path_equity_precision_llm_folder + '/llm training/epl05_output_Unattributable.csv', index=False)